In [15]:
#import libraries here!
import tweepy
import pandas as pd
import numpy as np

### 1. Twitter Authentification

In [2]:
consumer_key=""
consumer_secret=""

access_token="-"
access_token_secret=""

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [4]:
tweets = []
createdTime = []

### 2. Crawl the first set of tweets
* DateTime: 10:30 ~ 11:00 31 Jan 2018

In [ ]:
for i in range(10):
    for tweet in tweepy.Cursor(api.search,
                           q="btc",
                           rpp=5,
                           reseult_type="recent",
                           include_entities=True,
                           lang="en").items():
        print(tweet.created_at)
        tweets.append(tweet.text)
        createdTime.append(tweet.created_at)
        
    time.sleep(60) # Delay for 1 minute

In [8]:
df = pd.DataFrame({'text':tweets, 'createdTime':createdTime})

,createdTime,text
0,2018-01-31 11:00:24,Price of 1 LTC to USD: $164.5 (Change: +0.47 %...
1,2018-01-31 11:00:23,RT @LunyrInc: Lots of exciting plans ahead for...
2,2018-01-31 11:00:23,RT @DankProject: Our #first post\nHope this pr...
3,2018-01-31 11:00:19,📢 BTC Avg price 10220.7 USD | % chg 🔼1h 0.61 🔽...
4,2018-01-31 11:00:18,RT @aelfblockchain: $ELF now is tradable agai...


* The total size of records: 1352 / Unique tweets except Retweets: 937

In [39]:
df.shape, len(pd.unique(df.createdTime))

((1352, 4), 937)

### 3. Simple Sentiment Analysis with the Twitter dataset

In [16]:
from textblob import TextBlob
import re

def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analyze_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

In [25]:
df['cleanText'] = np.array([clean_tweet(tweet) for tweet in df['text']])
df['label'] = np.array([ analyze_sentiment(tweet) for tweet in df['cleanText'] ])

In [42]:
#Clean Text
df.head(10)

,createdTime,text,label,cleanText
0,2018-01-31 11:00:24,Price of 1 LTC to USD: $164.5 (Change: +0.47 %...,0,Price of 1 LTC to USD 164 5 Change 0 47 Price ...
1,2018-01-31 11:00:23,RT @LunyrInc: Lots of exciting plans ahead for...,1,RT Lots of exciting plans ahead for Lunyr We v...
2,2018-01-31 11:00:23,RT @DankProject: Our #first post\nHope this pr...,1,RT Our first post Hope this project will succe...
3,2018-01-31 11:00:19,📢 BTC Avg price 10220.7 USD | % chg 🔼1h 0.61 🔽...,0,BTC Avg price 10220 7 USD chg 1h 0 61 24h 7 17...
4,2018-01-31 11:00:18,RT @aelfblockchain: $ELF now is tradable agai...,0,RT ELF now is tradable against BTC ETH amp USD on
5,2018-01-31 11:00:17,"RT @Liqui_Exchange: INS/BTC, INS/ETH and INS/U...",0,RT Exchange INS BTC INS ETH and INS USDT marke...
6,2018-01-31 11:00:17,#BTC price increased! Buying @ $10181.22 &amp;...,0,BTC price increased Buying 10181 22 amp Sellin...
7,2018-01-31 11:00:14,Our #empowerment course starting tomorrow in @...,1,Our empowerment course starting tomorrow in is...
8,2018-01-31 11:00:13,RT @BinanceNewsfeed: 7 days left\n1. Follow us...,1,RT 7 days left 1 Follow us 2 Retweet 3 Win 200...
9,2018-01-31 11:00:13,#SUB Price is 0.00008489 (+0.00000085) #BTC / ...,-1,SUB Price is 0 00008489 0 00000085 BTC 0 86226...


In [27]:
pos_tweets = [ tweet for index, tweet in enumerate(df['cleanText']) if df['label'][index] > 0]
neu_tweets = [ tweet for index, tweet in enumerate(df['cleanText']) if df['label'][index] == 0]
neg_tweets = [ tweet for index, tweet in enumerate(df['cleanText']) if df['label'][index] < 0]

### 4. The result of this analysis

* You can find the fractions of tweets labeled as each class

In [29]:
print("Percentage of positive tweets: {}%".format(len(pos_tweets)*100/len(df['text'])))
print("Percentage of neutral tweets: {}%".format(len(neu_tweets)*100/len(df['text'])))
print("Percentage of negative tweets: {}%".format(len(neg_tweets)*100/len(df['text'])))

Percentage of positive tweets: 40.014792899408285%
Percentage of neutral tweets: 50.66568047337278%
Percentage of negative tweets: 9.319526627218934%


* The actual trend of the market during the 30 mins was slightly bearish.

##### Positive Tweets

In [34]:
pos_tweets

['RT Lots of exciting plans ahead for Lunyr We ve been moving fast and things will look more impressive than ever a couple months',
 'RT Our first post Hope this project will success soon BTC ETH BCH XPR Token',
 'Our empowerment course starting tomorrow in is now full We still have a few spaces remaining on our cou',
 'RT 7 days left 1 Follow us 2 Retweet 3 Win 2000 XRP BITFINEX BINANCE BITTREX BITHUMB POLONIEX GDAX BTCC BITS',
 'RT Hawala Incentive for Stake is LIVE 0 041 rewards everyday claim your rewards every 3 days And HAPPY NEW YEAR',
 'RT Airdrop live BITUR BTC ETH Airdrop 1 is now live and start to tag 5 of your friends after retweeting this airdrop Go',
 'BTC 0 61 10220 Top last h CHAT 89 43 5235st BON 40 64 11008st MEC 23 93 728st DIM 2',
 'Bitcoin 10 200 7 27 800 00 High 11 085 Low 9 680 25 Volume 6948 BTC BTC bitcoin',
 'Top 10 Mentions on Twitter in the last hour BTC TRX ETH XRP NEO LTC ICX ADA LSK BCH',
 'Wed Jan 31 15 00 02 2018 hash is good hashrate 183 7 MH s Ethereu

##### Negative tweets

In [36]:
neg_tweets

['SUB Price is 0 00008489 0 00000085 BTC 0 862268 0 01297 USD Market rank is 106 substratum bitcoin blockchain',
 'Bitstamp 10170 3 Coinbase 10111 11 Kraken 10137 Average 10139 47',
 'RT You guys make my night Excuse my outrageous sense of humor If I tweet again before tomorrow I owe everyone 0 25 B',
 'ESP marketcap 15 338 127 1 509 BTC CoinMarketCap rank 421 espers cryptocurrency bitcoin altcoins stats coinmarketcap',
 '10 59 GMT Arbitrage opportunity has occurred The diff is 0 00000501 BTC 1 Buy GAME on poloniex 0 00032999 BT',
 'RT These 9000 BTC prices bring me back to the old days couple months ago when 10k looked near impossible BTC XVG ETH',
 'RT BTC Seeing some accumulation here Will be watching carefully I bought some yesterday at 10k and things moved against me bu',
 'BTC Seeing some accumulation here Will be watching carefully I bought some yesterday at 10k and things moved ag',
 'RT Zeepin lists on KuCoin giveaway Follow retweet this competition tweet then fill in the form

##### Neutral tweets

In [38]:
neu_tweets

['Price of 1 LTC to USD 164 5 Change 0 47 Price of 1 LTC to BTC 0 0162216 Change 0 37 litecoin LTC LTC',
 'BTC Avg price 10220 7 USD chg 1h 0 61 24h 7 17 7d 8 01 btc',
 'RT ELF now is tradable against BTC ETH amp USD on',
 'RT Exchange INS BTC INS ETH and INS USDT markets added Ecosystem',
 'BTC price increased Buying 10181 22 amp Selling 10187 73 btc eth etc neo dgb dash strat eos omg pay xrp xmr gas',
 '1 BTC Bitcoin quotes 10101 21 10141 42 Bitstamp 10110 19 10132 51 Kraken 31 23 31 30 10069 51 1017',
 'Exchange has been added You can trade SCT on Cryptopia btc bitcoin eth crypto sct soma steemit soma',
 'It s not over boys and girls BTC is still',
 'RT ELF Listed on and will be tradable against BTC ETH HSR as of 7PM SGT today',
 'RT tech Former Ripple Employees Create a Cryptocurrency Hedge Fund xrpthestandard xrp ripple Rippl',
 'Cotizaciones al 31 01 2018 08 00 AM Bitcoin BTC 56 775 989 Ethereum ETH 6 117 999 Litecoin LTC 913 036 Mon',
 'When ALTCOINS go on a run use to instantly